In [1]:
import jieba

In [2]:
import os

In [3]:
#将一个文件的内容分词，以空格隔开，生成一个字符串。
#gb18030是一种中文的编码方式，这里必须用。
def cut_words(file_dir):
    with open(file_dir,encoding='gb18030') as file_obj:
        file_context = file_obj.read()
        file_context = jieba.cut(file_context)
        file_context = ' '.join(file_context)
        return file_context

In [4]:
#将一个文件夹内所有的文件分此后放在列表里，列表的每个元素都是一个文件分词后的字符串，同时在另外一个列表里存储每个元素的标签信息。
def load_word(file_dir,file_label):
    contents = []
    labels = []
    filelists = os.listdir(file_dir)
    for filelist in filelists:
        contents.append(cut_words(file_dir + '/' + filelist))
        labels.append(file_label)
    return contents,labels

In [5]:
#加载训练数据
contents_female_train,labels_female_train = load_word('text classification/train/女性','女性')
contents_sport_train,labels_sport_train = load_word('text classification/train/体育','体育')

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\yjh\AppData\Local\Temp\jieba.cache
Loading model cost 0.913 seconds.
Prefix dict has been built succesfully.


In [6]:
#加载测试数据
contents_female_test,labels_female_test = load_word('text classification/test/女性','女性')
contents_sport_test,labels_sport_test = load_word('text classification/test/体育','体育')

In [7]:
#将两种类型的训练数据合并，放在一个列表里
contents_train = []
labels_train = []
contents_train = contents_female_train + contents_sport_train
labels_train = labels_female_train + labels_sport_train

In [8]:
#将两种类型的测试数据合并，放在一个列表里
contents_test = []
labels_test = []
contents_test = contents_female_test + contents_sport_test
labels_test = labels_female_test + labels_sport_test

In [9]:
#加载停用词。这里需要先编码后解码，应对中文乱码问题。
with open('text classification/stop/stopword.txt',encoding='utf-8') as file_obj:
    stop_words = file_obj.read()
    stop_words = stop_words.encode('utf-8').decode('utf-8-sig')
    stop_words = stop_words.split('\n')

In [10]:
#通过TfidVectorizer类将文本变成特征向量
'''max_df 参数用来描述单词在文档中的最高出现率。
假设 max_df=0.5，代表一个单词在 50% 的文档中都出现过了，那么它只携带了非常少的信息，因此就不作为分词统计。
'''
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(stop_words=stop_words,max_df=0.5)
features_train = tf.fit_transform(contents_train)
features_test = tf.transform(contents_test)

c:\users\yjh\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\feature_extraction\text.py:286: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ain', 'aren', 'couldn', 'didn', 'doesn', 'don', 'hadn', 'hasn', 'haven', 'isn', 'll', 'mon', 'shouldn', 've', 'wasn', 'weren', 'won', 'wouldn'] not in stop_words.
  sorted(inconsistent))


In [11]:
#生成多项式朴素贝叶斯分类器
'''
当 alpha=1 时，使用的是 Laplace 平滑。Laplace 平滑就是采用加 1 的方式，来统计没有出现过的单词的概率。
这样当训练样本很大的时候，加 1 得到的概率变化可以忽略不计，也同时避免了零概率的问题。

当 0<alpha<1 时，使用的是 Lidstone 平滑。
对于 Lidstone 平滑来说，alpha 越小，迭代次数越多，精度越高。
可以设置 alpha 为 0.001。

'''
from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB(alpha=0.001)

In [12]:
#对分类器进行训练
clf.fit(features_train,labels_train)

MultinomialNB(alpha=0.001, class_prior=None, fit_prior=True)

In [13]:
#评估分类器效果
clf.score(features_test,labels_test)

0.9673202614379085